# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 40 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
y = 120   # y 
z = 120   # z 
x = 30    #FRAMES

In [5]:
#data resizing - Resizing all the images, so we can have all the images in a specific size
def crop_resize_img(img):
    if img.shape[0] != img.shape[1]:
        img=img[0:120,10:150]
    resized_image = imresize(img, (y,z))
    return resized_image

In [6]:
#using percentile to deal with outliers in the data.
def normalize_image(img):
    #using percentile for normalization for images, as min-max is giving better results.
    # normalized_image= img - np.percentile(img,15)/ np.percentile(img,85) - np.percentile(img,15) 
    normalized_image= (img - np.min(img))/(np.max(img)- np.min(img))
    return normalized_image

In [7]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)]  #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    resized_image=crop_resize_img(image)
                    batch_data[folder,idx,:,:,0] = normalize_image(resized_image[:, : , 0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalize_image(resized_image[:, : , 1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalize_image(resized_image[:, : , 2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = normalize_image(resized_image[:, : , 0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalize_image(resized_image[:, : , 1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalize_image(resized_image[:, : , 2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [8]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [9]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
#total_frames = 30
nb_frames =  30  # lenght of the video frames
nb_channel = 3 # numbe rof channels in images 3 for color(RGB) and 1 for Gray

nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [10]:
from keras.optimizers import Adam
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [11]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [12]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [13]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

### Experiment 1

In [14]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


 2/17 [==>...........................] - ETA: 2:55 - loss: 3.0091 - categorical_accuracy: 0.3000

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


17/17 [==============================] - 166s 10s/step - loss: 2.0689 - categorical_accuracy: 0.2574 - val_loss: 1.5672 - val_categorical_accuracy: 0.2200

Epoch 00001: saving model to model_init_2020-08-1613_51_03.807948/model-00001-2.07802-0.25943-1.56721-0.22000.h5
Epoch 2/20
17/17 [==============================] - 49s 3s/step - loss: 1.5626 - categorical_accuracy: 0.2890 - val_loss: 1.4647 - val_categorical_accuracy: 0.3000

Epoch 00002: saving model to model_init_2020-08-1613_51_03.807948/model-00002-1.56258-0.28900-1.46467-0.30000.h5
Epoch 3/20
17/17 [==============================] - 45s 3s/step - loss: 1.4563 - categorical_accuracy: 0.3381 - val_loss: 1.4380 - val_categorical_accuracy: 0.3833

Epoch 00003: saving model to model_init_2020-08-1613_51_03.807948/model-00003-1.45866-0.33787-1.43800-0.38333.h5
Epoch 4/20
17/17 [==============================] - 41s 2s/step - loss: 1.4142 - categorical_accuracy: 0.3901 - val_loss: 1.3088 - val_categorical_accuracy: 0.5333

Epoch 0000

### Experiment 2

In [15]:
batch_size = 50
num_epochs = 10


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Source path =  Project_data/train ; batch size = 50
Project_data/val ; batch size = 50
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


12/17 [====================>.........] - ETA: 36s - loss: 0.7276 - categorical_accuracy: 0.7133

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


17/17 [==============================] - 101s 6s/step - loss: 0.7533 - categorical_accuracy: 0.7026 - val_loss: 0.7801 - val_categorical_accuracy: 0.7333

Epoch 00001: saving model to model_init_2020-08-1613_51_03.807948/model-00001-0.73373-0.71225-0.78012-0.73333.h5
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.7465 - categorical_accuracy: 0.7104 - val_loss: 0.7381 - val_categorical_accuracy: 0.7467

Epoch 00002: saving model to model_init_2020-08-1613_51_03.807948/model-00002-0.74649-0.71041-0.73811-0.74667.h5
Epoch 3/10
17/17 [==============================] - 32s 2s/step - loss: 0.6514 - categorical_accuracy: 0.7376 - val_loss: 0.7303 - val_categorical_accuracy: 0.7333

Epoch 00003: saving model to model_init_2020-08-1613_51_03.807948/model-00003-0.65138-0.73756-0.73027-0.73333.h5
Epoch 4/10
17/17 [==============================] - 32s 2s/step - loss: 0.6520 - categorical_accuracy: 0.7376 - val_loss: 0.6735 - val_categorical_accuracy: 0.7733

Epoch 00004

### Experiment 3

In [16]:
batch_size = 60
num_epochs = 20


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val Source path =  Project_data/train; batch size = 60
Epoch 1/20
 ; batch size = 60


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


17/17 [==============================] - 93s 5s/step - loss: 0.7039 - categorical_accuracy: 0.7634 - val_loss: 1.0189 - val_categorical_accuracy: 0.6286

Epoch 00001: saving model to model_init_2020-08-1613_51_03.807948/model-00001-0.50878-0.80826-1.01892-0.62857.h5
Epoch 2/20
17/17 [==============================] - 7s 423ms/step - loss: 0.5513 - categorical_accuracy: 0.7255 - val_loss: 1.1420 - val_categorical_accuracy: 0.6125

Epoch 00002: saving model to model_init_2020-08-1613_51_03.807948/model-00002-0.55132-0.72549-1.14200-0.61250.h5
Epoch 3/20
17/17 [==============================] - 8s 449ms/step - loss: 0.6794 - categorical_accuracy: 0.7059 - val_loss: 0.6371 - val_categorical_accuracy: 0.7500

Epoch 00003: saving model to model_init_2020-08-1613_51_03.807948/model-00003-0.67941-0.70588-0.63707-0.75000.h5
Epoch 4/20
17/17 [==============================] - 8s 471ms/step - loss: 0.8378 - categorical_accuracy: 0.6667 - val_loss: 0.8771 - val_categorical_accuracy: 0.6833

Epoch 

### Experiment 4

In [17]:
batch_size = 30
num_epochs = 20


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 30
Source path =  Project_data/train ; batch size = 30
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


 3/17 [====>.........................] - ETA: 1:05 - loss: 0.4817 - categorical_accuracy: 0.8444

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


17/17 [==============================] - 69s 4s/step - loss: 0.4914 - categorical_accuracy: 0.8176 - val_loss: 0.6764 - val_categorical_accuracy: 0.8000

Epoch 00001: saving model to model_init_2020-08-1613_51_03.807948/model-00001-0.49139-0.81765-0.67637-0.80000.h5
Epoch 2/20
17/17 [==============================] - 21s 1s/step - loss: 0.6627 - categorical_accuracy: 0.7957 - val_loss: 1.2112 - val_categorical_accuracy: 0.6333

Epoch 00002: saving model to model_init_2020-08-1613_51_03.807948/model-00002-0.50507-0.82258-1.21123-0.63333.h5
Epoch 3/20
17/17 [==============================] - 7s 420ms/step - loss: 0.5313 - categorical_accuracy: 0.8039 - val_loss: 0.6558 - val_categorical_accuracy: 0.8333

Epoch 00003: saving model to model_init_2020-08-1613_51_03.807948/model-00003-0.53132-0.80392-0.65577-0.83333.h5
Epoch 4/20
17/17 [==============================] - 8s 444ms/step - loss: 0.6087 - categorical_accuracy: 0.7647 - val_loss: 0.6610 - val_categorical_accuracy: 0.8333

Epoch 00

### Experiment 5

In [19]:
batch_size = 40
num_epochs = 10


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  Project_data/train ; batch size =Epoch 1/10 Project_data/val ; batch size = 40

 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


 2/17 [==>...........................] - ETA: 1:32 - loss: 0.5027 - categorical_accuracy: 0.8500

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


17/17 [==============================] - 87s 5s/step - loss: 0.5813 - categorical_accuracy: 0.8023 - val_loss: 1.2437 - val_categorical_accuracy: 0.6900

Epoch 00001: saving model to model_init_2020-08-1613_51_03.807948/model-00001-0.54407-0.81750-1.24374-0.69000.h5
Epoch 2/10
17/17 [==============================] - 49s 3s/step - loss: 0.5018 - categorical_accuracy: 0.8210 - val_loss: 0.6517 - val_categorical_accuracy: 0.7833

Epoch 00002: saving model to model_init_2020-08-1613_51_03.807948/model-00002-0.50180-0.82097-0.65172-0.78333.h5
Epoch 3/10
17/17 [==============================] - 45s 3s/step - loss: 0.5629 - categorical_accuracy: 0.7748 - val_loss: 0.6759 - val_categorical_accuracy: 0.8000

Epoch 00003: saving model to model_init_2020-08-1613_51_03.807948/model-00003-0.54764-0.78202-0.67588-0.80000.h5
Epoch 4/10
17/17 [==============================] - 41s 2s/step - loss: 0.4547 - categorical_accuracy: 0.8328 - val_loss: 0.5742 - val_categorical_accuracy: 0.8667

Epoch 00004:

### Experiment 6 - Final Model

In [20]:
batch_size = 40
num_epochs = 30


train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


 2/17 [==>...........................] - ETA: 1:36 - loss: 0.4868 - categorical_accuracy: 0.8500

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


17/17 [==============================] - 86s 5s/step - loss: 0.5970 - categorical_accuracy: 0.8027 - val_loss: 0.9128 - val_categorical_accuracy: 0.6500

Epoch 00001: saving model to model_init_2020-08-1613_51_03.807948/model-00001-0.53728-0.81900-0.91283-0.65000.h5
Epoch 2/30
17/17 [==============================] - 51s 3s/step - loss: 0.4854 - categorical_accuracy: 0.8338 - val_loss: 0.7295 - val_categorical_accuracy: 0.7500

Epoch 00002: saving model to model_init_2020-08-1613_51_03.807948/model-00002-0.48535-0.83376-0.72952-0.75000.h5
Epoch 3/30
17/17 [==============================] - 43s 3s/step - loss: 0.5988 - categorical_accuracy: 0.8061 - val_loss: 0.6023 - val_categorical_accuracy: 0.8000

Epoch 00003: saving model to model_init_2020-08-1613_51_03.807948/model-00003-0.58266-0.81199-0.60226-0.80000.h5
Epoch 4/30
17/17 [==============================] - 39s 2s/step - loss: 0.5055 - categorical_accuracy: 0.8080 - val_loss: 0.6366 - val_categorical_accuracy: 0.7833

Epoch 00004:

17/17 [==============================] - 37s 2s/step - loss: 0.5353 - categorical_accuracy: 0.8131 - val_loss: 0.6782 - val_categorical_accuracy: 0.8333

Epoch 00028: saving model to model_init_2020-08-1613_51_03.807948/model-00028-0.53533-0.81315-0.67816-0.83333.h5
Epoch 29/30
17/17 [==============================] - 35s 2s/step - loss: 0.4954 - categorical_accuracy: 0.8131 - val_loss: 0.6592 - val_categorical_accuracy: 0.7500

Epoch 00029: saving model to model_init_2020-08-1613_51_03.807948/model-00029-0.49544-0.81315-0.65921-0.75000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 9.313226188509272e-13.
Epoch 30/30
17/17 [==============================] - 36s 2s/step - loss: 0.5015 - categorical_accuracy: 0.8062 - val_loss: 0.6229 - val_categorical_accuracy: 0.8500

Epoch 00030: saving model to model_init_2020-08-1613_51_03.807948/model-00030-0.50154-0.80623-0.62290-0.85000.h5
